In [1]:
!pip install pandas numpy matplotlib pyarrow scikit-learn tkan temporal_linear_network scipy "jax[cuda12]" seaborn keras_sig

## Dynamic VWAP paper all results

Runned using an 3 RTX 4090 + AMD EPYC 9654

This notebook was used to run all the results displayed in the paper, but as the session disconnected the notebook itself didn't registered, so only the graphs and table where saved and are not present in the notebook. 
You can obtain same results are the one in the paper rerunning this notebook still.

In [2]:
import os
import time

BACKEND = 'jax'  # You can use any backend here
os.environ['KERAS_BACKEND'] = BACKEND

import numpy as np
import pandas as pd

from sklearn.metrics import r2_score

import seaborn as sns
import matplotlib.pyplot as plt

import keras
from keras import ops
from keras.initializers import Initializer
from keras import Model, Sequential
from keras.layers import Dense, LSTM, Input, Reshape

from tkan import TKAN
from tln import TLN

from keras_sig import SigLayer

import keras 
from keras import ops
from keras import Model, Input
from keras.layers import Layer, LSTM, Dense, Input, Add, LayerNormalization, Multiply, Reshape, Activation, TimeDistributed, Flatten, Lambda, MultiHeadAttention, Concatenate, Dropout
from keras_efficient_kan import KANLinear


keras.utils.set_random_seed(1) 

early_stopping_callback = lambda : keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.00001,
    patience=13,
    mode="min",
    restore_best_weights=True,
    start_from_epoch=0,
)
lr_callback = lambda : keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.25,
    patience=6,
    mode="min",
    min_delta=0.00001,
    min_lr=0.000025,
    verbose=0,
)
callbacks = lambda : [early_stopping_callback(), lr_callback(), keras.callbacks.TerminateOnNaN()]


# Initialize data parallel distribution
# It will automatically detect all available GPU devices
data_parallel = keras.distribution.DataParallel()

# Set the global distribution
keras.distribution.set_distribution(data_parallel)

import jax
N_DEVICES = len(jax.devices())

N_MAX_EPOCHS = 1000
N_RUNS = 3

NOTIONAL_PARQUET_PATH = "hourly_notionals.parquet"
VOLUME_PARQUET_PATH = "hourly_volumes.parquet"
CLOSES_PARQUET_PATH = "hourly_closes.parquet"

sorted_assets = list((pd.read_parquet(VOLUME_PARQUET_PATH)==0).sum(axis=0).sort_values().index)
N_ASSETS = 80
all_assets = sorted_assets[:N_ASSETS]
train_assets = sorted_assets[:N_ASSETS][::2] # 1 over 2
print('training on', train_assets)
print()
print('testing on', all_assets)
jax.devices()

training on ['BTC', 'BCH', 'EOS', 'TRX', 'LINK', 'ADA', 'DASH', 'XTZ', 'BNB', 'IOTA', 'VET', 'QTUM', 'THETA', 'ZIL', 'ZRX', 'OMG', 'SXP', 'BAND', 'MKR', 'DOT', 'YFI', 'CRV', 'TRB', 'RUNE', 'EGLD', 'STORJ', 'UNI', 'FTM', 'ENJ', 'KSM', 'FIL', 'LRC', 'MATIC', 'BEL', 'ALPHA', 'SKL', '1INCH', 'SAND', 'LIT', 'UNFI']

testing on ['BTC', 'ETH', 'BCH', 'XRP', 'EOS', 'LTC', 'TRX', 'ETC', 'LINK', 'XLM', 'ADA', 'XMR', 'DASH', 'ZEC', 'XTZ', 'ATOM', 'BNB', 'ONT', 'IOTA', 'BAT', 'VET', 'NEO', 'QTUM', 'IOST', 'THETA', 'ALGO', 'ZIL', 'KNC', 'ZRX', 'COMP', 'OMG', 'DOGE', 'SXP', 'KAVA', 'BAND', 'RLC', 'MKR', 'SNX', 'DOT', 'DEFI', 'YFI', 'BAL', 'CRV', 'WAVES', 'TRB', 'SUSHI', 'RUNE', 'SOL', 'EGLD', 'ICX', 'STORJ', 'BLZ', 'UNI', 'AVAX', 'FTM', 'FLM', 'ENJ', 'REN', 'KSM', 'NEAR', 'FIL', 'AAVE', 'LRC', 'RSR', 'MATIC', 'OCEAN', 'BEL', 'AXS', 'ALPHA', 'ZEN', 'SKL', 'GRT', '1INCH', 'CHZ', 'SAND', 'ANKR', 'LIT', 'CTK', 'UNFI', 'REEF']


[CudaDevice(id=0)]

In [3]:
def prepare_data_without_ahead_inputs(vwaps: np.ndarray, volumes: np.ndarray, features: np.ndarray, lookback: int, n_ahead: int, autoscale_target: bool = True):
    n_row = vwaps.shape[0]
    X = {'prices': np.zeros((n_row - lookback - n_ahead, lookback, 1)), 'volumes': np.zeros((n_row - lookback - n_ahead, lookback, 1)), 'features': np.zeros((n_row - lookback - n_ahead, lookback, features.shape[1]))}
    y = {'prices': np.zeros((n_row - lookback - n_ahead, n_ahead, 1)), 'volumes': np.zeros((n_row - lookback - n_ahead, n_ahead, 1))}
    offset=0
    print('hir?')
    for row in range(lookback, n_row - n_ahead):
        X[row - lookback-offset] = features[row - lookback:row]
        if autoscale_target:
            s=np.sum(volumes[row - lookback:row])
            if s>0 and np.sum(volumes[row:row + n_ahead])>0:
                y['prices'][row - lookback-offset] = np.expand_dims(vwaps[row:row + n_ahead] / vwaps[row - lookback], axis=-1)
                y['volumes'][row - lookback-offset] = np.expand_dims(volumes[row:row + n_ahead] / s, axis=-1)
            else:
                
                offset+=1
        else:
            y['prices'][row - lookback] = np.expand_dims(vwaps[row:row + n_ahead], axis=-1)
            y['volumes'][row - lookback] = np.expand_dims(volumes[row:row + n_ahead], axis=-1)
    y = np.concatenate([y['volumes'], y['prices']], axis=-1)
    
    return X[:-offset], y[:-offset]

def prepare_data_with_ahead_inputs(vwaps: np.ndarray, volumes: np.ndarray, features: np.ndarray, lookback: int, n_ahead: int, autoscale_target: bool = True):
    n_row = vwaps.shape[0]
    X = np.zeros((n_row - lookback - n_ahead, lookback + n_ahead - 1, features.shape[1]))
    y = {'prices': np.zeros((n_row - lookback - n_ahead, n_ahead, 1)), 'volumes': np.zeros((n_row - lookback - n_ahead, n_ahead, 1))}
    offset=0
    for row in range(lookback, n_row - n_ahead):
        X[row - lookback-offset] = features[row - lookback:row + n_ahead - 1]
        if autoscale_target:
            s=np.sum(volumes[row - lookback:row])
            if s>0 and np.sum(volumes[row:row + n_ahead])>0:
                y['prices'][row - lookback-offset] = np.expand_dims(vwaps[row:row + n_ahead] / vwaps[row - lookback], axis=-1)
                y['volumes'][row - lookback-offset] = np.expand_dims(volumes[row:row + n_ahead] / s, axis=-1)
            else:
                offset+=1
        else:
            y['prices'][row - lookback] = np.expand_dims(vwaps[row:row + n_ahead], axis=-1)
            y['volumes'][row - lookback] = np.expand_dims(volumes[row:row + n_ahead], axis=-1)
    y = np.concatenate([y['volumes'], y['prices']], axis=-1)
    return X[:-offset-1], y[:-offset-1]


def full_generate(target_asset, lookback = 120, n_ahead = 12, test_split = 0.2, include_ahead_inputs = False, autoscale_target=True):
    volumes = pd.read_parquet(VOLUME_PARQUET_PATH).astype(np.float32)
    notionals = pd.read_parquet(NOTIONAL_PARQUET_PATH).astype(np.float32)
    assets = [target_asset]
    
    assert target_asset in assets
    volumes = volumes[assets].dropna()
    notionals = notionals[assets].dropna()
    notionals = notionals.loc[volumes.index]
    volumes = volumes.loc[notionals.index]
    vwaps = pd.DataFrame(notionals.values / volumes.values, index=volumes.index, columns = volumes.columns)
    vwaps = vwaps.ffill().dropna()
    notionals = notionals.loc[vwaps.index]
    volumes = volumes.loc[vwaps.index]
    #Create feature
    features = volumes / volumes.shift(lookback + n_ahead).rolling(24 * 7 * 2).mean()
    features['hour'] = volumes.index.hour
    features['dow'] = volumes.index.dayofweek
    for asset in assets:
        features[f'returns {asset}'] = vwaps[asset] / vwaps[asset].shift() - 1.
    
    #Create volume and prices
    volumes = volumes[target_asset]
    vwaps = vwaps[target_asset]
    
    #Remove NaN and align
    features = features.loc[volumes.index].dropna()
    volumes = volumes.loc[features.index]
    vwaps = vwaps.ffill().loc[volumes.index]

    if include_ahead_inputs:
        X, y = prepare_data_with_ahead_inputs(vwaps.values, volumes.values, features.values, lookback, n_ahead, autoscale_target =  autoscale_target)
    else:
        X, y = prepare_data_without_ahead_inputs(vwaps.values, volumes.values, features.values, lookback, n_ahead, autoscale_target = autoscale_target)
    
    test_row = int(len(y) * (1 - test_split))
    
    X_train = X[:test_row]
    X_test = X[test_row:]
    
    y_train, y_test = y[:test_row], y[test_row:]
    
    return X_train, X_test, y_train, y_test

# Losses

In [4]:
def quadratic_vwap_loss(y_true, y_pred):
    # Ensure all operations are done with float32
    vwap_achieved = keras.ops.sum(y_pred[..., 0] * y_true[..., 1], axis = 1) / keras.ops.sum(y_pred[..., 0], axis = 1)
    vwap_mkt = keras.ops.sum(y_true[..., 0] * y_true[..., 1], axis = 1) / keras.ops.sum(y_true[..., 0], axis = 1)
    vwap_diff = vwap_achieved / vwap_mkt - 1.
    loss = keras.ops.mean(keras.ops.square(vwap_diff))
    return loss

def absolute_vwap_loss(y_true, y_pred):
    # Ensure all operations are done with float32
    vwap_achieved = keras.ops.sum(y_pred[..., 0] * y_true[..., 1], axis = 1) / keras.ops.sum(y_pred[..., 0], axis = 1)
    vwap_mkt = keras.ops.sum(y_true[..., 0] * y_true[..., 1], axis = 1) / keras.ops.sum(y_true[..., 0], axis = 1)
    vwap_diff = vwap_achieved / vwap_mkt - 1.
    loss = keras.ops.mean(keras.ops.abs(vwap_diff))
    return loss

def volume_curve_loss(y_true, y_pred):
    # Ensure all operations are done with float32
    volume_curve_achieved = y_pred[..., 0] / keras.ops.sum(y_pred[..., 0], axis = 1, keepdims=True)
    volume_curve_mkt = y_true[..., 0] / keras.ops.sum(y_true[..., 0], axis = 1, keepdims=True)
    volume_curve_diff = volume_curve_achieved - volume_curve_mkt
    loss = keras.ops.mean(keras.ops.square(volume_curve_diff))
    return loss

# Static VWAP model

In [5]:
@keras.utils.register_keras_serializable(name="StaticVWAP")
class StaticVWAP(Model):
    def __init__(self, lookback, n_ahead, input_include_aheads = False, internal_model = None):
        super(StaticVWAP, self).__init__()
        self.lookback = lookback
        self.n_ahead = n_ahead
        self.input_include_aheads = input_include_aheads 
        self.internal_model = internal_model if internal_model is not None else TLN(output_len = n_ahead,
                            output_features = 1,
                            flatten_output = False,
                            hidden_layers = 2,
                            use_convolution = True,
                            name = 'TLN')

    def build(self, input_shape):
        if isinstance(input_shape, dict):
            feature_shape = input_shape
            self.dictionnary_input = True
        else:
            feature_shape = input_shape
            self.dictionnary_input = False
        internal_model_input_shape = (feature_shape[0], self.lookback, feature_shape[2])
        internal_model_target_output_shape = (feature_shape[0], self.n_ahead, 1)
        self.internal_model.build(internal_model_input_shape)
        internal_model_output_shape = self.internal_model.compute_output_shape(internal_model_input_shape)
        if internal_model_output_shape != internal_model_target_output_shape:
            if (*internal_model_output_shape, 1) ==  internal_model_target_output_shape:
                self.add_dims_to_internal_model = True
            else:
                raise ValueError(f"""internal model do not respect required output shape:
                Received inputs shape {internal_model_input_shape} 
                Return outputs shape {internal_model_output_shape}
                Required outputs shape {internal_model_target_output_shape}
                """)
        else:
            self.add_dims_to_internal_model = False

    def call(self, inputs):
        if self.dictionnary_input:
            features = inputs
        else:
            features = inputs
        if self.input_include_aheads:
            features = keras.ops.slice(features, (0,0,0),(features.shape[0], self.lookback, features.shape[2]))
        preds = self.internal_model(features)
        if self.add_dims_to_internal_model:
            preds = keras.ops.expand_dims(preds, axis=-1)
        volume_curve = keras.ops.softmax(preds, axis=1)
        results = keras.ops.concatenate([volume_curve, keras.ops.zeros_like(preds)], axis=-1)
        return results
    
    def get_config(self):
        config = super().get_config()
        config.update({
            'lookback': self.lookback,
            'n_ahead': self.n_ahead,
            'input_include_aheads': self.input_include_aheads,
            'internal_model': {
                'class_name': self.internal_model.__class__.__name__,
                'config': self.internal_model.get_config()
            }
        })
        return config
    
    @classmethod
    def from_config(cls, config):
        # Extract internal model config
        internal_model_config = config.pop('internal_model')
        internal_model_class = globals()[internal_model_config['class_name']]
        internal_model = internal_model_class.from_config(internal_model_config['config'])
        
        # Create new instance with remaining config
        return cls(
            lookback=config['lookback'],
            n_ahead=config['n_ahead'],
            input_include_aheads=config['input_include_aheads'],
            internal_model=internal_model
        )

# Dynamic VWAP model

In [6]:
@keras.utils.register_keras_serializable(name="EqualInitializer")
class EqualInitializer(Initializer):
    """Initializes weights to 1/n_ahead."""
    
    def __init__(self, n_ahead):
        self.n_ahead = n_ahead
        
    def __call__(self, shape, dtype=None):
        return ops.ones(shape, dtype=dtype) / self.n_ahead

        
    def get_config(self):
        return {'n_ahead': self.n_ahead}


@keras.utils.register_keras_serializable(name="PositiveSumToOneConstraint")
class PositiveSumToOneConstraint(keras.constraints.Constraint):
    """Constrains the weights to be positive and sum to 1."""
    
    def __call__(self, w):
        # First ensure values are positive
        w = keras.ops.maximum(w, 0)
        # Then normalize to sum to 1
        return w / (keras.ops.sum(w) + keras.backend.epsilon())

    def get_config(self):
        return {}


@keras.utils.register_keras_serializable(name="DynamicVWAP")
class DynamicVWAP(Model):
    def __init__(self, lookback, n_ahead, hidden_size=100, hidden_rnn_layer=2, *args, **kwargs):
        super(DynamicVWAP, self).__init__(*args, **kwargs)
        self.lookback = lookback
        self.n_ahead = n_ahead
        self.hidden_size = hidden_size
        self.hidden_rnn_layer = hidden_rnn_layer
        
    def build(self, input_shape):
        feature_shape = input_shape
        assert feature_shape[1] == self.lookback + self.n_ahead - 1
        self.internal_rnn = Sequential([
            TKAN(self.hidden_size, return_sequences=True)
            for _ in range(self.hidden_rnn_layer)
        ])
        self.internal_rnn.build(feature_shape)    
        internal_model_output_shape = self.internal_rnn.compute_output_shape(feature_shape)
        self.internal_hidden_to_volume = [
            Sequential([
                Dense(self.hidden_size, activation='relu'),
                Dense(self.hidden_size, activation='relu'),
                Dense(1, activation='tanh') 
            ])
            for _ in range(self.n_ahead - 1)
        ]
        for i in range(self.n_ahead - 1):
            self.internal_hidden_to_volume[i].build((feature_shape[0], self.hidden_size + i))
        self.base_volume_curve = self.add_weight(
            shape=(self.n_ahead,),
            name="base_curve",
            initializer=EqualInitializer(self.n_ahead),
            constraint=PositiveSumToOneConstraint(),  # Using the new constraint
            trainable=True
        )
        
    def call(self, inputs):
        features = inputs
        rnn_hidden = self.internal_rnn(features)
        total_volume = ops.zeros((features.shape[0],1))
        
        for t in range(0, self.n_ahead - 1):
            if t>0:
                current_hidden = ops.concatenate([rnn_hidden[:, self.lookback + t, :], volume_curve], axis=1)
            else:
                current_hidden = rnn_hidden[:, self.lookback + t, :]
            estimated_factor = 1. + self.internal_hidden_to_volume[t](current_hidden)
            estimated_volume = self.base_volume_curve[t] * estimated_factor 
            estimated_volume = keras.ops.clip(estimated_volume, 0., 1. - total_volume)
            total_volume += estimated_volume
            if t==0:
                volume_curve=estimated_volume
            else:
                volume_curve=ops.concatenate([volume_curve, estimated_volume], axis=1)
            
        estimated_volume=1. - total_volume
        volume_curve=ops.concatenate([volume_curve, estimated_volume], axis=1)
        volume_curve=ops.expand_dims(volume_curve, axis=2)
        results = keras.ops.concatenate([volume_curve, keras.ops.zeros_like(volume_curve)], axis=-1)
        return results

    def get_config(self):
        config = super().get_config()
        config.update({
            'lookback': self.lookback,
            'n_ahead': self.n_ahead,
            'hidden_size': self.hidden_size,
            'hidden_rnn_layer': self.hidden_rnn_layer,
        })
        return config

    def compute_output_shape(self, input_shape):
        return (input_shape[0], 2, self.n_ahead)

In [7]:
@keras.utils.register_keras_serializable(name="AddAndNorm")
class AddAndNorm(Layer):
    def __init__(self, **kwargs):
        super(AddAndNorm, self).__init__(**kwargs)

    def build(self, input_shape):
        self.add_layer = Add()
        self.add_layer.build(input_shape)
        self.norm_layer = LayerNormalization()
        self.norm_layer.build(self.add_layer.compute_output_shape(input_shape))
    
    def call(self, inputs):
        tmp = self.add_layer(inputs)
        tmp = self.norm_layer(tmp)
        return tmp

    def compute_output_shape(self, input_shape):
        return input_shape[0]  # Assuming all input shapes are the same

    def get_config(self):
        config = super().get_config()
        return config


@keras.utils.register_keras_serializable(name="GRN")
class Gate(Layer):
    def __init__(self, hidden_layer_size = None, **kwargs):
        super(Gate, self).__init__(**kwargs)
        self.hidden_layer_size = hidden_layer_size
        

    def build(self, input_shape):
        if self.hidden_layer_size is None:
            self.hidden_layer_size = input_shape[-1]
        self.dense_layer = Dense(self.hidden_layer_size)
        self.gated_layer = Dense(self.hidden_layer_size, activation='sigmoid')
        self.dense_layer.build(input_shape)
        self.gated_layer.build(input_shape)
        self.multiply = Multiply()

    def call(self, inputs):
        dense_output = self.dense_layer(inputs)
        gated_output = self.gated_layer(inputs)
        return ops.multiply(dense_output, gated_output)

    def compute_output_shape(self, input_shape):
        return input_shape[:-1] + (self.hidden_layer_size,)

    def get_config(self):
        config = super().get_config()
        config.update({
            'hidden_layer_size': self.hidden_layer_size,
        })
        return config


@keras.utils.register_keras_serializable(name="GRN")
class GRN(Layer):
    def __init__(self, hidden_layer_size, output_size=None, **kwargs):
        super(GRN, self).__init__(**kwargs)
        self.hidden_layer_size = hidden_layer_size
        self.output_size = output_size

    def build(self, input_shape):
        if self.output_size is None:
            self.output_size = self.hidden_layer_size
        self.skip_layer = Dense(self.output_size)
        self.skip_layer.build(input_shape)
        
        self.hidden_layer_1 = Dense(self.hidden_layer_size, activation='elu')
        self.hidden_layer_1.build(input_shape)
        self.hidden_layer_2 = Dense(self.hidden_layer_size)
        self.hidden_layer_2.build((*input_shape[:2], self.hidden_layer_size))
        self.gate_layer = Gate(self.output_size)
        self.gate_layer.build((*input_shape[:2], self.hidden_layer_size))
        self.add_and_norm_layer = AddAndNorm()
        self.add_and_norm_layer.build([(*input_shape[:2], self.output_size),(*input_shape[:2], self.output_size)])

    def call(self, inputs):
        skip = self.skip_layer(inputs)
        hidden = self.hidden_layer_1(inputs)
        hidden = self.hidden_layer_2(hidden)
        gating_output = self.gate_layer(hidden)
        return self.add_and_norm_layer([skip, gating_output])

    def compute_output_shape(self, input_shape):
        return input_shape[:-1] + (self.output_size,)

    def get_config(self):
        config = super().get_config()
        config.update({
            'hidden_layer_size': self.hidden_layer_size,
            'output_size': self.output_size,
        })
        return config


@keras.utils.register_keras_serializable(name="VariableSelectionNetwork")
class VariableSelectionNetwork(Layer):
    def __init__(self, num_hidden, **kwargs):
        super(VariableSelectionNetwork, self).__init__(**kwargs)
        self.num_hidden = num_hidden

    def build(self, input_shape):
        batch_size, time_steps, embedding_dim, num_inputs = input_shape
        self.softmax = Activation('softmax')
        self.num_inputs = num_inputs
        self.flatten_dim = time_steps * embedding_dim * num_inputs
        self.reshape_layer = Reshape(target_shape=[time_steps, embedding_dim * num_inputs])
        self.reshape_layer.build(input_shape)
        self.mlp_dense = GRN(hidden_layer_size = self.num_hidden, output_size=num_inputs)
        self.mlp_dense.build((batch_size, time_steps, embedding_dim * num_inputs))
        self.grn_layers = [GRN(self.num_hidden) for _ in range(num_inputs)]
        for i in range(num_inputs):
            self.grn_layers[i].build(input_shape[:3])
        super(VariableSelectionNetwork, self).build(input_shape)

    def call(self, inputs):
        _, time_steps, embedding_dim, num_inputs = inputs.shape
        flatten = self.reshape_layer(inputs)
        # Variable selection weights
        mlp_outputs = self.mlp_dense(flatten)
        sparse_weights = keras.activations.softmax(mlp_outputs)
        sparse_weights = ops.expand_dims(sparse_weights, axis=2)
        
        # Non-linear Processing & weight application
        trans_emb_list = []
        for i in range(num_inputs):
            grn_output = self.grn_layers[i](inputs[:, :, :, i])
            trans_emb_list.append(grn_output)
        
        transformed_embedding = ops.stack(trans_emb_list, axis=-1)
        combined = ops.multiply(sparse_weights, transformed_embedding)
        temporal_ctx = ops.sum(combined, axis=-1)
        
        return temporal_ctx

    def get_config(self):
        config = super().get_config()
        config.update({
            'num_hidden': self.num_hidden,
        })
        return config


@keras.utils.register_keras_serializable(name="EmbeddingLayer")
class EmbeddingLayer(Layer):
    def __init__(self, num_hidden, **kwargs):
        super(EmbeddingLayer, self).__init__(**kwargs)
        self.num_hidden = num_hidden

    def build(self, input_shape):
        self.dense_layers = [
            Dense(self.num_hidden) for _ in range(input_shape[-1])
        ]
        for i in range(input_shape[-1]):
            self.dense_layers[i].build((*input_shape[:2], 1))
        super(EmbeddingLayer, self).build(input_shape)

    def call(self, inputs):
        embeddings = [dense_layer(inputs[:, :, i:i+1]) for i, dense_layer in enumerate(self.dense_layers)]
        return ops.stack(embeddings, axis=-1)

    def compute_output_shape(self, input_shape):
        return list(input_shape[:-1]) + [self.num_hidden, input_shape[-1]]

    def get_config(self):
        config = super().get_config()
        config.update({
            'num_hidden': self.num_hidden,
        })
        return config

@keras.utils.register_keras_serializable(name="DynamicVWAPTransformer")
class DynamicVWAPTransformer(Model):
    def __init__(self, lookback, n_ahead, hidden_size=100, hidden_rnn_layer=2, num_heads=3, num_embedding=3, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.lookback = lookback
        self.n_ahead = n_ahead
        self.hidden_size = hidden_size
        self.hidden_rnn_layer = hidden_rnn_layer
        self.num_embedding = num_embedding
        self.num_heads = num_heads
        
    def build(self, input_shape):
        feature_shape = input_shape
        assert feature_shape[1] == self.lookback + self.n_ahead - 1

        self.embedding = EmbeddingLayer(self.num_embedding)
        self.embedding.build(input_shape)
        embedding_output_shape = self.embedding.compute_output_shape(input_shape)
        self.vsn = VariableSelectionNetwork(self.hidden_size)
        self.vsn.build(embedding_output_shape)
        vsn_output_shape = (input_shape[0], input_shape[1], self.hidden_size)
        
        # RNN layers
        self.internal_rnn = Sequential([
            TKAN(self.hidden_size, return_sequences=True)
            for _ in range(self.hidden_rnn_layer)
        ])
        self.internal_rnn.build(vsn_output_shape)
        self.gate = Gate()
        self.gate.build(vsn_output_shape)
        self.addnorm = AddAndNorm()
        self.addnorm.build([vsn_output_shape,vsn_output_shape])
        self.grn = GRN(self.hidden_size)
        self.grn.build(vsn_output_shape)
        # Multi-head attention layer
        self.attention = MultiHeadAttention(
            num_heads=self.num_heads,
            key_dim=self.hidden_size // self.num_heads,
            value_dim=self.hidden_size // self.num_heads,
            use_bias=True
        )
        self.attention.build(vsn_output_shape, vsn_output_shape, vsn_output_shape)#as the tkan do not changes shapes
        
        # Dense layers for volume prediction
        self.internal_hidden_to_volume = [
            Sequential([
                Dense(self.hidden_size, activation='relu'),
                Dense(self.hidden_size, activation='relu'),
                Dense(1, activation='tanh')
            ])
            for _ in range(self.n_ahead - 1)
        ]
        
        for i in range(self.n_ahead - 1):
            self.internal_hidden_to_volume[i].build((feature_shape[0], self.hidden_size + i))
            
        # Base volume curve
        self.base_volume_curve = self.add_weight(
            shape=(self.n_ahead,),
            name="base_curve",
            initializer=EqualInitializer(self.n_ahead),
            constraint=PositiveSumToOneConstraint(),
            trainable=True
        )
        super(DynamicVWAPTransformer, self).build(input_shape)
        
    def call(self, inputs):
        embedded_features = self.embedding(inputs)
        selected = self.vsn(embedded_features)
        # Get RNN hidden states
        rnn_hidden = self.internal_rnn(selected)
        all_context = self.addnorm([self.gate(rnn_hidden), selected])
        enriched = self.grn(all_context)
        
        # Apply causal self-attention
        attended_hidden = self.attention(
            query=enriched,
            value=enriched,
            key=enriched,
            use_causal_mask=True
        )
        
        total_volume = ops.zeros((inputs.shape[0], 1))
        
        for t in range(0, self.n_ahead - 1):
            if t > 0:
                current_hidden = ops.concatenate([attended_hidden[:, self.lookback + t, :], volume_curve], axis=1)
            else:
                current_hidden = attended_hidden[:, self.lookback + t, :]
                
            estimated_factor = 1. + self.internal_hidden_to_volume[t](current_hidden)
            estimated_volume = self.base_volume_curve[t] * estimated_factor
            estimated_volume = keras.ops.clip(estimated_volume, 0., 1. - total_volume)
            total_volume += estimated_volume
            
            if t == 0:
                volume_curve = estimated_volume
            else:
                volume_curve = ops.concatenate([volume_curve, estimated_volume], axis=1)
        
        estimated_volume = 1. - total_volume
        volume_curve = ops.concatenate([volume_curve, estimated_volume], axis=1)
        volume_curve = ops.expand_dims(volume_curve, axis=2)
        results = keras.ops.concatenate([volume_curve, keras.ops.zeros_like(volume_curve)], axis=-1)
        
        return results
    
    def get_config(self):
        config = super().get_config()
        config.update({
            'lookback': self.lookback,
            'n_ahead': self.n_ahead,
            'hidden_size': self.hidden_size,
            'hidden_rnn_layer': self.hidden_rnn_layer,
            'num_embedding': self.num_embedding,
            'num_heads': self.num_heads
        })
        return config
        
    def compute_output_shape(self, input_shape):
        return (input_shape[0], 2, self.n_ahead)
    


# Graph Functions

In [8]:
def create_aggregated_pred_graph(name, n_ahead, preds, absolute_model_score, quadratic_model_score, r2_volume_curve):
    # Set the style for a more academic look
    sns.set_style("whitegrid")
    sns.set_context("paper", font_scale=1.2)
    
    # Create a DataFrame for easier plotting
    df = pd.DataFrame(preds[:,:,0].T, columns=[f'Execution_{i+1}' for i in range(preds.shape[0])])
    df['Period'] = range(1, n_ahead + 1)
    df_melted = df.melt('Period', var_name='Execution', value_name='Allocation')
    
    # Create the plot
    plt.figure(figsize=(10, 6))
    
    # Use a more subtle color palette
    palette = sns.color_palette("Blues", n_colors=preds.shape[0])
    sns.lineplot(data=df_melted, x='Period', y='Allocation', hue='Execution', 
                 palette=palette, alpha=0.3, legend=False)
    
    # Calculate and plot the mean allocation
    mean_allocation = df_melted.groupby('Period')['Allocation'].mean()
    sns.lineplot(x=mean_allocation.index, y=mean_allocation.values, 
                 color='red', linewidth=2, label='Mean Allocation')
    
    # Customize the plot
    plt.title(f'VWAP Execution Allocation per Period using {name} over {n_ahead} step', fontsize=16)
    plt.xlabel('Period', fontsize=14)
    plt.ylabel('Allocation', fontsize=14)
    plt.ylim(0, df_melted['Allocation'].max() * 1.1)  # Set y-axis limit with some headroom
    
    # Add a legend
    plt.legend(title='', loc='upper right', frameon=True)
    
    # Add text annotations for performance metrics
    text = (f"Abs. VWAP Loss (x1e2): {1e2*absolute_model_score:.4f}\n"
            f"Quad. VWAP Loss (x1e4): {1e4*quadratic_model_score:.4f}\n"
            f"Volume Curve R²: {r2_volume_curve:.4f}")
    plt.text(0.02, 0.98, text, transform=plt.gca().transAxes, 
             verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # Adjust layout and save
    plt.tight_layout()
    plt.savefig(f'vwap_execution_allocation_{name}_{n_ahead}_steps.png', dpi=300, bbox_inches='tight')
    plt.show()

# Full results generation function

In [9]:

data_parallel = keras.distribution.DataParallel()
keras.distribution.set_distribution(data_parallel)
BATCH_SIZE = 1024 * N_DEVICES
lookback = 60
sig_lookback = 400
n_ahead = 12
validation_split = 0.2

optimization_function = absolute_vwap_loss
full_X_train = None
for asset in train_assets:
    X_train, X_test, y_train, y_test = full_generate(asset, lookback=lookback, n_ahead=n_ahead, include_ahead_inputs = True)
    val_size = int(len(X_train) * validation_split)
    X_val = X_train[-val_size:]
    y_val = y_train[-val_size:]
    X_train = X_train[:-val_size]
    y_train = y_train[:-val_size]
    if full_X_train is None:
        full_X_train = X_train
        full_X_val = X_val
        full_X_test = X_test
        full_y_train = y_train
        full_y_val = y_val
        full_y_test = y_test
    else:
        full_X_train = np.concatenate([full_X_train, X_train], axis=0)
        full_y_train = np.concatenate([full_y_train, y_train], axis=0)
        full_y_val = np.concatenate([full_y_val, y_val], axis=0)
        full_y_test = np.concatenate([full_y_test, y_test], axis=0)

val_samples = full_X_val.shape[0]
val_batches = val_samples // BATCH_SIZE
val_samples_adjusted = val_batches * BATCH_SIZE
full_X_val = full_X_val[:val_samples_adjusted]
full_y_val = full_y_val[:val_samples_adjusted]


train_samples = full_X_train.shape[0]
train_batches = train_samples // BATCH_SIZE
train_samples_adjusted = train_batches * BATCH_SIZE
full_X_train = full_X_train[:train_samples_adjusted]
full_y_train = full_y_train[:train_samples_adjusted]


/tmp/ipykernel_93748/3040326385.py:55: RuntimeWarning: invalid value encountered in divide
  vwaps = pd.DataFrame(notionals.values / volumes.values, index=volumes.index, columns = volumes.columns)
/tmp/ipykernel_93748/3040326385.py:55: RuntimeWarning: invalid value encountered in divide
  vwaps = pd.DataFrame(notionals.values / volumes.values, index=volumes.index, columns = volumes.columns)
/tmp/ipykernel_93748/3040326385.py:55: RuntimeWarning: invalid value encountered in divide
  vwaps = pd.DataFrame(notionals.values / volumes.values, index=volumes.index, columns = volumes.columns)
/tmp/ipykernel_93748/3040326385.py:55: RuntimeWarning: invalid value encountered in divide
  vwaps = pd.DataFrame(notionals.values / volumes.values, index=volumes.index, columns = volumes.columns)
/tmp/ipykernel_93748/3040326385.py:55: RuntimeWarning: invalid value encountered in divide
  vwaps = pd.DataFrame(notionals.values / volumes.values, index=volumes.index, columns = volumes.columns)
/tmp/ipykernel_

In [10]:
keras.backend.clear_session()
data_parallel = keras.distribution.DataParallel()
keras.distribution.set_distribution(data_parallel)


train_times_dynamic=[]
train_times_dynamic_sig=[]

for run in range(N_RUNS):
    print('do run', run)

    if not os.path.exists(f'base_dynamic_transformer_model_200_{run}.keras'):
        base_dynamic_sig_model = DynamicVWAPTransformer(lookback=lookback, n_ahead=n_ahead, hidden_size=200)
        base_dynamic_sig_model.compile(optimizer='adam', loss=optimization_function)
        t1=time.time()
        history = base_dynamic_sig_model.fit(full_X_train, full_y_train, batch_size=BATCH_SIZE, epochs=N_MAX_EPOCHS, 
                            validation_data=(full_X_val, full_y_val), callbacks=callbacks(), shuffle=True, verbose=True)
        train_times_dynamic_sig.append(time.time()-t1)
        base_dynamic_sig_model.save(f'base_dynamic_transformer_model_200_{run}.keras')
        del base_dynamic_sig_model


print('mean train_times_dynamic', np.mean(train_times_dynamic), 'std', np.std(train_times_dynamic))
print('mean train_times_dynamic', np.mean(train_times_dynamic_sig), 'std', np.std(train_times_dynamic_sig))

do run 0
do run 1
do run 2
mean train_times_dynamic nan std nan
mean train_times_dynamic nan std nan


/usr/local/lib/python3.10/dist-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/_core/_methods.py:223: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.10/dist-packages/numpy/_core/_methods.py:181: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/usr/local/lib/python3.10/dist-packages/numpy/_core/_methods.py:215: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [11]:
def create_full_comparison(assets, train_assets, lookback, n_ahead, sig_lookback, verbose=True):

    BATCH_SIZE = 128 * N_DEVICES
    results = {}
    loaded_model = {}
    optimization_function = absolute_vwap_loss
    for target_asset in assets:
        for model_type in ["Naive", 'Globally Fitted DynamicVWAP Transformer']:

            if model_type not in loaded_model:
                loaded_model[model_type] = {}
            
            X_train, X_test, y_train, y_test = full_generate(target_asset, lookback=sig_lookback, n_ahead=n_ahead, include_ahead_inputs = True)
            X_train = X_train[:,-lookback-n_ahead+1:]
            X_test = X_test[:,-lookback-n_ahead+1:]

            val_size = int(len(X_train) * validation_split)
            X_val = X_train[-val_size:]
            y_val = y_train[-val_size:]
            X_train = X_train[:-val_size]
            y_train = y_train[:-val_size]
            
            val_samples = X_val.shape[0]
            val_batches = val_samples // BATCH_SIZE
            val_samples_adjusted = val_batches * BATCH_SIZE
            X_val = X_val[:val_samples_adjusted]
            y_val = y_val[:val_samples_adjusted]
            
            train_samples = X_train.shape[0]
            train_batches = train_samples // BATCH_SIZE
            train_samples_adjusted = train_batches * BATCH_SIZE
            X_train = X_train[:train_samples_adjusted]
            y_train = y_train[:train_samples_adjusted]

            results_absolute = []
            results_quadratic = []
            results_volume_curve_r2 = []
            training_times = []
            all_preds = np.zeros((y_test.shape[0] * N_RUNS, *y_test.shape[1:]))
            for run in range(N_RUNS):
                training_time = 0
                data_parallel = keras.distribution.DataParallel()
                keras.distribution.set_distribution(data_parallel)
                if model_type=='Globally Fitted DynamicVWAP Transformer':
                    keras.distribution.set_distribution(None)
                    if run not in loaded_model[model_type]:
                        loaded_model[model_type][run] = keras.models.load_model(f'base_dynamic_transformer_model_200_{run}.keras', compile=False)

                    model = loaded_model[model_type][run]
                    
                    naive_preds = np.copy(y_test)
                    naive_preds[:,:,0] = 1/n_ahead
                    preds = model.predict(X_test, verbose=False)
                    
                    absolute_model_score = absolute_vwap_loss(y_test, preds)
                    quadratic_model_score = quadratic_vwap_loss(y_test, preds)
                    r2_volume_curve = r2_score(y_true=(y_test[:,:,0] / np.sum(y_test[:,:,0], axis=1, keepdims=True)).flatten(), 
                                               y_pred=preds[:,:,0].flatten())
                elif model_type=='Naive':
                    if optimization_function!=absolute_vwap_loss or run!=0:
                        break
                    model = None
                    training_time = 0
                    preds = np.copy(y_test)
                    preds[:,:,0] = 1./n_ahead
                    absolute_model_score = absolute_vwap_loss(y_test, preds)
                    quadratic_model_score = quadratic_vwap_loss(y_test, preds)
                    r2_volume_curve = r2_score(y_true=(y_test[:,:,0] / np.sum(y_test[:,:,0], axis=1, keepdims=True)).flatten(), y_pred=preds[:,:,0].flatten())
                
                else:
                    raise ValueError
                
                all_preds[run * y_test.shape[0]:(run + 1) * y_test.shape[0]] = preds
                results_absolute.append(absolute_model_score)
                results_quadratic.append(quadratic_model_score)
                results_volume_curve_r2.append(r2_volume_curve)
                training_times.append(training_time)
                
                del model
    
            if model_type not in ["Asset Fitted DynamicVWAP", "Asset Fitted DynamicSigVWAP","Globally Fitted DynamicVWAP Transformer", "Globally Fitted DynamicSigVWAP"] and optimization_function!=absolute_vwap_loss:
                continue
            if model_type == 'Naive':
                all_preds = all_preds[:y_test.shape[0]]
                

            opt_function_name = optimization_function.__name__

            name = f'{model_type} {"using " + opt_function_name if model_type in ["Asset Fitted DynamicVWAP", "Asset Fitted DynamicSigVWAP","Globally Fitted DynamicVWAP Transformer", "Globally Fitted DynamicSigVWAP"] else ""}: {target_asset}'
            #create_aggregated_pred_graph(name, n_ahead, all_preds , absolute_model_score, quadratic_model_score, r2_volume_curve)
            
            results_absolute = np.array(results_absolute) *1e2
            results_quadratic = np.array(results_quadratic) *1e4
            results_volume_curve_r2 = np.array(results_volume_curve_r2)
            
            results[f'{model_type}_{target_asset}_{opt_function_name}'] = {
                "model type": model_type,
                "asset": target_asset,
                "Training Asset": target_asset in train_assets, 
                "optimization_function": opt_function_name if model_type in ["Asset Fitted DynamicVWAP", "Asset Fitted DynamicSigVWAP","Globally Fitted DynamicVWAP Transformer", "Globally Fitted DynamicSigVWAP"] else "N/A",
                "mean_absolute_vwap_loss": np.mean(results_absolute),
                "std_absolute_vwap_loss": np.std(results_absolute) if model_type in ["Asset Fitted DynamicVWAP", "Asset Fitted DynamicSigVWAP","Globally Fitted DynamicVWAP Transformer", "Globally Fitted DynamicSigVWAP"] else 0,
                "mean_quadratic_vwap_loss": np.mean(results_quadratic),
                "std_quadratic_vwap_loss": np.std(results_quadratic) if model_type in ["Asset Fitted DynamicVWAP", "Asset Fitted DynamicSigVWAP","Globally Fitted DynamicVWAP Transformer", "Globally Fitted DynamicSigVWAP"] else 0,
                "mean_r2_volume_curve": np.mean(results_volume_curve_r2),
                "std_r2_volume_curve": np.std(results_volume_curve_r2) if model_type in ["Asset Fitted DynamicVWAP", "Asset Fitted DynamicSigVWAP","Globally Fitted DynamicVWAP Transformer", "Globally Fitted DynamicSigVWAP"] else 0,
                "mean_training_time": np.mean(training_times),
                "std_training_time": np.std(training_times) if model_type in ["Asset Fitted DynamicVWAP", "Asset Fitted DynamicSigVWAP","Globally Fitted DynamicVWAP Transformer", "Globally Fitted DynamicSigVWAP"] else 0,
            }
            if verbose:
                print("Is Train asset: ", target_asset in train_assets)
                print(f"{model_type}_{target_asset}_{opt_function_name}: Mean Training time: {np.mean(training_times):.4f} (±{np.std(training_times):.4f})")
                print(f"{model_type}_{target_asset}_{opt_function_name}: Mean absolute VWAP loss (1e-2): {np.mean(results_absolute):.4f} (±{np.std(results_absolute):.4f})")
                print(f"{model_type}_{target_asset}_{opt_function_name}: Mean quadratic VWAP loss (1e-4): {np.mean(results_quadratic):.4f} (±{np.std(results_quadratic):.4f})")
                print(f"{model_type}_{target_asset}_{opt_function_name}: Mean R2 on volume curve: {np.mean(results_volume_curve_r2):.4f} (±{np.std(results_volume_curve_r2):.4f})")

    # Convert results to a DataFrame
    results_df = pd.DataFrame(results.values())
    
    return results_df


results_df = create_full_comparison(assets = all_assets, train_assets = train_assets, lookback = lookback, n_ahead = n_ahead, sig_lookback = sig_lookback, verbose=True)
display(results_df)
results_df.to_csv(f'full_results_200_transformer_{lookback}_{n_ahead}.csv')

Is Train asset:  True
Naive_BTC_absolute_vwap_loss: Mean Training time: 0.0000 (±0.0000)
Naive_BTC_absolute_vwap_loss: Mean absolute VWAP loss (1e-2): 0.1600 (±0.0000)
Naive_BTC_absolute_vwap_loss: Mean quadratic VWAP loss (1e-4): 0.0889 (±0.0000)
Naive_BTC_absolute_vwap_loss: Mean R2 on volume curve: 0.0000 (±0.0000)
Is Train asset:  True
Globally Fitted DynamicVWAP Transformer_BTC_absolute_vwap_loss: Mean Training time: 0.0000 (±0.0000)
Globally Fitted DynamicVWAP Transformer_BTC_absolute_vwap_loss: Mean absolute VWAP loss (1e-2): 0.1035 (±0.0006)
Globally Fitted DynamicVWAP Transformer_BTC_absolute_vwap_loss: Mean quadratic VWAP loss (1e-4): 0.0415 (±0.0005)
Globally Fitted DynamicVWAP Transformer_BTC_absolute_vwap_loss: Mean R2 on volume curve: -0.3438 (±0.0638)


/tmp/ipykernel_93748/3040326385.py:55: RuntimeWarning: invalid value encountered in divide
  vwaps = pd.DataFrame(notionals.values / volumes.values, index=volumes.index, columns = volumes.columns)


Is Train asset:  False
Naive_ETH_absolute_vwap_loss: Mean Training time: 0.0000 (±0.0000)
Naive_ETH_absolute_vwap_loss: Mean absolute VWAP loss (1e-2): 0.1791 (±0.0000)
Naive_ETH_absolute_vwap_loss: Mean quadratic VWAP loss (1e-4): 0.1176 (±0.0000)
Naive_ETH_absolute_vwap_loss: Mean R2 on volume curve: 0.0000 (±0.0000)


/tmp/ipykernel_93748/3040326385.py:55: RuntimeWarning: invalid value encountered in divide
  vwaps = pd.DataFrame(notionals.values / volumes.values, index=volumes.index, columns = volumes.columns)


Is Train asset:  False
Globally Fitted DynamicVWAP Transformer_ETH_absolute_vwap_loss: Mean Training time: 0.0000 (±0.0000)
Globally Fitted DynamicVWAP Transformer_ETH_absolute_vwap_loss: Mean absolute VWAP loss (1e-2): 0.1150 (±0.0025)
Globally Fitted DynamicVWAP Transformer_ETH_absolute_vwap_loss: Mean quadratic VWAP loss (1e-4): 0.0548 (±0.0070)
Globally Fitted DynamicVWAP Transformer_ETH_absolute_vwap_loss: Mean R2 on volume curve: -0.3820 (±0.0474)


...

/tmp/ipykernel_93748/3040326385.py:55: RuntimeWarning: invalid value encountered in divide
  vwaps = pd.DataFrame(notionals.values / volumes.values, index=volumes.index, columns = volumes.columns)


Is Train asset:  False
Naive_REEF_absolute_vwap_loss: Mean Training time: 0.0000 (±0.0000)
Naive_REEF_absolute_vwap_loss: Mean absolute VWAP loss (1e-2): 0.3821 (±0.0000)
Naive_REEF_absolute_vwap_loss: Mean quadratic VWAP loss (1e-4): 0.7124 (±0.0000)
Naive_REEF_absolute_vwap_loss: Mean R2 on volume curve: 0.0000 (±0.0000)


/tmp/ipykernel_93748/3040326385.py:55: RuntimeWarning: invalid value encountered in divide
  vwaps = pd.DataFrame(notionals.values / volumes.values, index=volumes.index, columns = volumes.columns)


Is Train asset:  False
Globally Fitted DynamicVWAP Transformer_REEF_absolute_vwap_loss: Mean Training time: 0.0000 (±0.0000)
Globally Fitted DynamicVWAP Transformer_REEF_absolute_vwap_loss: Mean absolute VWAP loss (1e-2): 0.3247 (±0.0012)
Globally Fitted DynamicVWAP Transformer_REEF_absolute_vwap_loss: Mean quadratic VWAP loss (1e-4): 0.6876 (±0.0057)
Globally Fitted DynamicVWAP Transformer_REEF_absolute_vwap_loss: Mean R2 on volume curve: -0.5286 (±0.0139)


,model type,asset,Training Asset,optimization_function,mean_absolute_vwap_loss,std_absolute_vwap_loss,mean_quadratic_vwap_loss,std_quadratic_vwap_loss,mean_r2_volume_curve,std_r2_volume_curve,mean_training_time,std_training_time
0,Naive,BTC,True,N/A,0.160043,0.000000,0.088911,0.000000,0.000000,0.000000,0.0,0.0
1,Globally Fitted DynamicVWAP Transformer,BTC,True,absolute_vwap_loss,0.103502,0.000642,0.041549,0.000493,-0.343816,0.063788,0.0,0.0
2,Naive,ETH,False,N/A,0.179140,0.000000,0.117632,0.000000,0.000000,0.000000,0.0,0.0
3,Globally Fitted DynamicVWAP Transformer,ETH,False,absolute_vwap_loss,0.114969,0.002529,0.054774,0.006967,-0.381977,0.047374,0.0,0.0
4,Naive,BCH,True,N/A,0.295928,0.000000,0.382944,0.000000,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
155,Globally Fitted DynamicVWAP Transformer,CTK,False,absolute_vwap_loss,0.232764,0.003321,0.247628,0.013193,-0.403369,0.006088,0.0,0.0
156,Naive,UNFI,True,N/A,0.416821,0.000000,0.967334,0.000000,0.000000,0.000000,0.0,0.0
157,Globally Fitted DynamicVWAP Transformer,UNFI,True,absolute_vwap_loss,0.357463,0.007359,0.743137,0.030235,-0.529852,0.007943,0.0,0.0
158,Naive,REEF,False,N/A,0.382102,0.000000,0.712378,0.000000,0.000000,0.000000,0.0,0.0
